In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/csa/CSA3')
os.getcwd()

'/content/drive/MyDrive/csa/CSA3'

In [3]:
def loadmodel_set(load_epoch):
  class Opion():
    def __init__(self):
        self.dataroot= r'I:\irregular holes\paris_eval_gt' #image dataroot
        self.maskroot= r'I:\irregular holes\testing_mask_dataset'#mask dataroot
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# seconde stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'.\checkpoints' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch='6'
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=True
  from util.data_load import Data_load
  from models.models import create_model
  import torch
  import os
  import torchvision
  from torch.utils import data
  import torchvision.transforms as transforms
  opt = Opion()
  model = create_model(opt)
  total_steps = 0

  model.load(load_epoch)
  print('load model successfully!')
  return model

In [4]:
def loadimg(data_path, mask_path):
  from util.data_load import Data_load
  import torchvision.transforms as transforms
  from torch.utils import data
  transform_mask = transforms.Compose(
      [transforms.Resize((256,256)),
      transforms.ToTensor(),
      ])
  transform = transforms.Compose(
      [
      transforms.Resize((256,256)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

  dataset_train = Data_load(data_path, mask_path, transform, transform_mask)
  iterator_train = (data.DataLoader(dataset_train, batch_size=1))
  print(len(dataset_train))
  return iterator_train

In [5]:
def inpaint(iterator_test,model,height,width):
  import time
  import torch
  import torchvision
  import os
  import numpy as np
  import torchvision.transforms as transforms

  from skimage.metrics import structural_similarity as SSIM
  from skimage.metrics import peak_signal_noise_ratio as PSNR
  from skimage.metrics import mean_squared_error as MSE
  import cv2

  total_psnr = [0]*200
  total_ssim = [0]*200
  total_mse = [0]*200
  save_dir = "./content/fin"
  if os.path.exists(save_dir) is False:
      os.makedirs(save_dir)

  epoch=1
  i=0
  for image, mask in (iterator_test):
      iter_start_time = time.time()
      image=image.cuda()
      mask=mask.cuda()
      mask=mask[0][0]
      mask=torch.unsqueeze(mask,0)
      mask=torch.unsqueeze(mask,1)
      mask=mask.byte()

      model.set_input(image,mask)
      model.set_gt_latent()
      model.test()
      print("fix")
      real_A,real_B,fake_B=model.get_current_visuals()
      pic = (torch.cat([real_A, real_B,fake_B], dim=0) + 1) / 2.0
      torchvision.utils.save_image(pic, '%s/Epoch_(%d).jpg' % (
      save_dir, epoch), nrow=1)
      #fake_B = (torch.cat([fake_B], dim=0) + 1) / 2.0
      retransform = transforms.Compose(
        [
        transforms.Resize((height,width))]
        )
      fake_B = retransform(fake_B)
      fake_B = (fake_B * 0.5) + 0.5

      real_B = retransform(real_B)
      #real_B = (real_B * 0.5) + 0.5
      real_B = (torch.cat([real_B], dim=0) + 1) / 2.0

      real_A = retransform(real_A)
      #real_B = (real_B * 0.5) + 0.5
      real_A = (torch.cat([real_A], dim=0) + 1) / 2.0

      torchvision.utils.save_image(fake_B, '%s/re.jpg' % (
      save_dir), nrow=1)
      torchvision.utils.save_image(real_B, '%s/re1.jpg' % (
      save_dir), nrow=1)
      torchvision.utils.save_image(real_A, '%s/break.jpg' % (
      save_dir), nrow=1)

      #参数评价
      #img_cp1 = cv2.imread("/content/fin/re.jpg")
      #img_cp2 = cv2.imread("/content/fin/re1.jpg")
      #psnr = PSNR(img_cp1, img_cp2)
      #ssim = SSIM(img_cp1, img_cp2, multichannel=True, channel_axis=2)
      #mse = MSE(img_cp1, img_cp2)
      #total_psnr[i] = psnr
      #total_ssim[i] = ssim
      #total_mse[i] = mse
      #i=i+1
  #print("psnr")
  #print(np.mean(total_psnr))
  #print("ssim")
  #print(np.mean(total_ssim))
  #print("mse")
  #print(np.mean(total_mse))
  return fake_B



In [6]:
import cv2
import numpy as np
import PIL
from PIL import Image, ImageOps
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
img_path = "./testimg/real/test0.jpg"
nomask_path = "./testimg/nomask/0.png"
IMAGE_UPLOAD_PATH = "./testimg/real"
MASK_PATH = "./testimg/mask1"
noMASK_PATH = "./testimg/mask"
save_path = "./testimg/fin"


def restore_image(model, image_or, mask=None):
    if mask is not None:
      #mask = mask['layers'][0]
      data_path = IMAGE_UPLOAD_PATH
      mask_path = MASK_PATH
      iterator_test = loadimg(data_path, mask_path)
      #model = loadmodel_set(30)
      repaired_image=inpaint(iterator_test,model)
      width = image_or.width
      height = image_or.height
      retransform = transforms.Compose(
        [
        transforms.Resize((height,width))]
        )
      repaired_image = retransform(repaired_image)
      repaired_image = (repaired_image * 0.5) + 0.5
    else:
      #mask = Image.new("L", (512, 512), color=0)
      #cv2.imwrite(nomask_path, mask)
      #mask.save(nomask_path)
      data_path = IMAGE_UPLOAD_PATH
      mask_path = noMASK_PATH
      iterator_test = loadimg(data_path, mask_path)
      #model = loadmodel_set(30)
      repaired_image=inpaint(iterator_test,model)
      width = image_or.width
      height = image_or.height
      retransform = transforms.Compose(
        [transforms.RandomHorizontalFlip(),
        transforms.Resize((height,width))]
        )
      repaired_image = retransform(repaired_image)
      repaired_image = (repaired_image * 0.5) + 0.5
    return repaired_image

def restore_image1(model, image_or, mask):
  iterator_test = loadimg(image_or, mask)
  width = image_or.width
  height = image_or.height
  repaired_image=inpaint(iterator_test,model,height,width)
  #torchvision.utils.save_image(repaired_image, '%s/re.jpg' % (save_path), nrow=1)
  repaired_image = Image.open("/content/fin/re.jpg")
  #return repaired_image
  #torchvision.utils.save_image(repaired_image, '%s/re.jpg' % (save_path), nrow=1)
  return repaired_image
def restore_image2(model, image_or, mask):
  iterator_test = loadimg(image_or, mask)
  repaired_image=inpaint(iterator_test,model,218,178)
  repaired_image = Image.open("./content/fin/re.jpg")
  return repaired_image

In [ ]:
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import mean_squared_error as MSE
import cv2

def get_spm(img_cp1, img_cp2):

    psnr = PSNR(img_cp1, img_cp2)
    ssim = SSIM(img_cp1, img_cp2, multichannel=True, channel_axis=2)
    mse = MSE(img_cp1, img_cp2)
    return psnr, ssim, mse

#img_cp1 = cv2.imread("./testimg/fin/re.jpg")
#img_cp2 = cv2.imread("./testimg/real/test.jpg")
img_cp1 = cv2.imread("/content/fin/re.jpg")
img_cp2 = cv2.imread("/content/fin/re1.jpg")
psnr, ssim, mse = get_spm(img_cp1, img_cp2)
print("PSNR:{}\nSSIM:{}\nMSE:{}".format(psnr, ssim, mse))

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
model = loadmodel_set(6)
#model_CA = loadmodel_set(30)
#model_SH = loadmodel_set(32)
#model_CSAF = loadmodel_set(2)

In [ ]:
img = restore_image2(model_CA,image_path,mask_path)

NameError: name 'model_CA' is not defined

In [ ]:
img = restore_image2(model_SH,image_path,mask_path)

1
psnr
0.13977687910702202
ssim
0.004689703636456226
mse
0.520610332268151


<ipython-input-13-b43277043ffe>:64: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = SSIM(img_cp1, img_cp2, multichannel=True, channel_axis=2)


In [ ]:
img = restore_image2(model_CSAF,image_path,mask_path)

1
psnr
0.14923164714764361
ssim
0.004622178607701302
mse
0.3368351630416108


<ipython-input-5-b43277043ffe>:64: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = SSIM(img_cp1, img_cp2, multichannel=True, channel_axis=2)


In [ ]:
#现成图片直接测试
image_path = "./testimg/real"
mask_path = "./testimg/mask"
img = restore_image2(model,image_path,mask_path)
#现场测试
image_or = Image.open("./testimg/real/test.jpg")
mask = Image.open("./testimg/mask/00276.png")
image_path = "/content/img/uploaded_image.jpg"
IMAGE_PATH = "/content/img"
mask_path = "/content/mask/mask_image.jpg"
MASK_PATH = "/content/mask"
save_path = "/content/fin"
NOMASK_PATH = "/content/nomask"
nomask_path = "/content/nomask/nomask.jpg"
#image_or = Image.open(image_path)
mask = "./testimg/mask"
image_or = "./testimg/real"
#img = restore_image1(model,image_or,mask)
#torchvision.utils.save_image(img, '%s/re.jpg' % (save_path), nrow=1)

2


<ipython-input-43-913535d19d2e>:56: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = SSIM(img_cp1, img_cp2, multichannel=True, channel_axis=2)


psnr
0.8310472880326607
ssim
0.019810540851973896
mse
0.125250146033055


In [7]:
!pip install gradio==4.28.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.8 MB

In [8]:
import gradio as gr
import cv2
import numpy as np
import PIL
from PIL import Image, ImageOps
import os


title_center = "<h1 style='text-align: center;'>基于深度学习的图像修复</h1>"


image_path = "./content/img/uploaded_image.jpg"
IMAGE_PATH = "./content/img"
mask_path = "./content/mask/mask_image.png"
MASK_PATH = "./content/mask"
save_path = "./content/fin"
NOMASK_PATH = "./content/nomask"
nomask_path = "./content/nomask/nomask.jpg"
if os.path.exists(IMAGE_PATH) is False:
    os.makedirs(IMAGE_PATH)
if os.path.exists(MASK_PATH) is False:
    os.makedirs(MASK_PATH)
if os.path.exists(save_path) is False:
    os.makedirs(save_path)
if os.path.exists(NOMASK_PATH) is False:
    os.makedirs(NOMASK_PATH)

In [9]:
model_use = loadmodel_set(6)

csa_net


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 153MB/s] 
/content/drive/MyDrive/csa/CSA3/models/CSA.py:25: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda'

initialize network with normal


/content/drive/MyDrive/csa/CSA3/models/networks.py:53: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
/content/drive/MyDrive/csa/CSA3/models/networks.py:63: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorCSA(
  (model): UnetSkipConnectionBlock_3(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3(
            (model): Sequential(
              (0): LeakyReLU(negative_slope=0.2, inplace=True)
  

In [10]:
import time
def restore_image(image_or, mask):
      image_or = cv2.cvtColor(image_or,cv2.COLOR_BGR2RGB)
      cv2.imwrite(image_path, image_or)
      image_or = Image.open("./content/img/uploaded_image.jpg")
      mask = mask['layers'][0]
      mask = cv2.cvtColor(mask,cv2.COLOR_BGR2RGB)
      cv2.imwrite(mask_path, mask)
      iterator_test = loadimg(IMAGE_PATH, MASK_PATH)
      width = image_or.width
      height = image_or.height
      model = model_use
      repaired_image=inpaint(iterator_test,model,height,width)
      repaired_image = Image.open("./content/fin/re.jpg")
      mask = Image.open("./content/mask/mask_image.png")
      break_image = Image.open("./content/fin/break.jpg")
      #torchvision.utils.save_image(repaired_image, '%s/re.jpg' % (save_path), nrow=1)
      return repaired_image,break_image, mask
def restore_image1(image_or):
      image_or = cv2.cvtColor(image_or,cv2.COLOR_BGR2RGB)
      cv2.imwrite(image_path, image_or)
      image_or = Image.open("./content/img/uploaded_image.jpg")
      iterator_test = loadimg(IMAGE_PATH, NOMASK_PATH)
      width = image_or.width
      height = image_or.height
      model = model_use
      repaired_image=inpaint(iterator_test,model,height,width)
      fin_image = Image.open("./content/fin/re.jpg")
      break_image = Image.open("./content/fin/break.jpg")

      #return repaired_image
      #torchvision.utils.save_image(repaired_image, '%s/re.jpg' % (save_path), nrow=1)
      return fin_image,break_image

def restore_image3(image_or, mask):
      image_or = cv2.cvtColor(image_or,cv2.COLOR_BGR2RGB)
      cv2.imwrite(image_path, image_or)
      image_or = Image.open("./content/img/uploaded_image.jpg")
      cv2.imwrite(mask_path, mask)
      iterator_test = loadimg(IMAGE_PATH, MASK_PATH)
      width = image_or.width
      height = image_or.height
      model = model_use
      repaired_image=inpaint(iterator_test,model,height,width)
      fin_image = Image.open("./content/fin/re.jpg")
      break_image = Image.open("./content/fin/break.jpg")

      #return repaired_image
      #torchvision.utils.save_image(repaired_image, '%s/re.jpg' % (save_path), nrow=1)
      return fin_image,break_image

def te(inputs,image):
  image = image['layers'][0]
  return inputs,image

def restore_image000(image_or):
      fin_image = Image.open("./content/fin/re.jpg")
      return fin_image

gr.close_all()

iface1 = gr.Interface(
    fn=restore_image1,
    inputs=gr.Image(label="上传要修复的图像"),
    outputs=["image","image"],
    title=title_center,
    description="随机损坏修复",
    )
iface2 = gr.Interface(
    fn=restore_image,
    inputs=[
        gr.Image(label="上传要修复的图像"),
        gr.ImageEditor(label="涂改mask（可选）")
    ],
    outputs=["image","image","image"],
    title=title_center,
    description="涂改损坏修复",
    )

iface3 = gr.Interface(
    fn=restore_image3,
    inputs=[
        gr.Image(label="上传要修复的图像"),
        gr.Image(label="mask")
    ],
    outputs=["image","image"],
    title=title_center,
    description="涂改损坏修复",
    )

tabbed_interface = gr.TabbedInterface([iface1, iface2, iface3], ["随机损坏修复", "涂改损坏修复", "指定损坏"])
tabbed_interface.launch()

IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------
IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------
IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://aa02c45ff9e0aae0ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
